<a href="https://colab.research.google.com/github/amanteur/TDA_Cover_detection/blob/main/Practics/TDA_CoverDetection_Practics_2_Supervised_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.2.0
!pip install gudhi

     |████████████████████████████████| 9.9 MB 4.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.2.0 which is incompatible.


     |████████████████████████████████| 28.2 MB 22 kB/s 


In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import gudhi as gd
import gudhi.representations as gdr
from tqdm import tqdm 

#Extracting data

In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/CourseWork/pers_feat_datasets/'

In [ ]:
df_dict = {}
for _, _, files in os.walk(root_dir):
  for file in files:
    name = file.split('.')[0]
    df = pd.read_json(root_dir + file)
    df_dict [name] = df

#Test/train

In [ ]:
def split_data(df_dict, label_col='name', test_size=0.2, random_state=42):
  #extracting labels aka songs' names
  labels = set(df_dict[np.random.choice(list(df_dict))][label_col].tolist())
  #making train/test labels
  lbl_train, lbl_test = train_test_split(list(labels), test_size=test_size, random_state=random_state)

  train_test_dict = {}
  for name, df in df_dict.items():
    train_df = df[df[label_col].isin(lbl_train)]
    test_df = df[df[label_col].isin(lbl_test)]

    train_test_dict.setdefault(name,{})
    train_test_dict[name]['train'] = train_df
    train_test_dict[name]['test'] = test_df

  return train_test_dict

In [ ]:
train_test_dict = split_data(df_dict, test_size=0.15)
#example
train_test_dict['af_orig']['train'].shape, train_test_dict['af_orig']['test'].shape 

((140, 7), (24, 7))

#Making pairs for train

In [ ]:
def make_pairs(train_test_dict):
  pairs_dict = {}
  for name, train_test in train_test_dict.items():

    pairs_dict.setdefault(name,{})

    for tt, df in train_test.items():
      if tt == 'test':
        continue
      #resetting index in order to get id of song
      part_df = df.reset_index()
      pair = pd.merge(part_df, part_df, how='cross')
      #exclude rows of same songs
      pair = pair[pair.index_x != pair.index_y]

      #delete same rows
      pair['indices'] = list(zip(pair.index_x, pair.index_y))
      pair[['indices']] = pair[['indices']].applymap(frozenset)
      pair = pair.drop_duplicates(subset=['indices'])
      
      #adding predicting label
      pair['ref_cover'] = np.where((pair['name_x'] == pair['name_y']), 1, 0)

      #normalizing dataframe (because there are not so much "1" labels)
      pos_idx = pair.index[pair['ref_cover'] == 1]
      n_pos = pos_idx.shape[0]
      neg_idx = pair.index[pair['ref_cover'] == 0]
      np.random.seed(0)
      neg_idx_stay = np.random.choice(neg_idx, n_pos, replace=False)

      indices = np.concatenate([pos_idx, neg_idx_stay])

      output_pair = pair[pair.index.isin(indices)]

      #dropping unnecessary columns
      output_pair.drop(labels=['index_x', 'index_y', 'indices'], inplace=True, axis=1)

      #adding to dict
      pairs_dict[name] = output_pair.reset_index(drop=True)

  return pairs_dict

In [ ]:
pairs_df_dict = make_pairs(train_test_dict)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [ ]:
pairs_df_dict['af_DTM']

,name_x,persistence_DTM_x,Landscape_x,Silhouette_x,Entropy_vector_x,Betti_curves_x,name_y,persistence_DTM_y,Landscape_y,Silhouette_y,Entropy_vector_y,Betti_curves_y,ref_cover
0,Blue_Collar_Man,"[[0.1774890428, 0.2075652335], [0.1992981601, ...","[[0.0028132621, 0.0056265242, 0.0084397862, 0....","[[0.0, 9.81306e-05, 0.00019626110000000002, 0....","[[0.3193268741, 0.3193268741, 0.3193268741, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Blue_Collar_Man,"[[0.14879702990000002, 0.174270759], [0.161238...","[[0.0026977515, 0.0053955031, 0.0080932546, 0....","[[0.0, 0.0001046298, 0.00020925970000000002, 0...","[[0.3615991845, 0.3615991845, 0.3615991845, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",1
1,Blue_Collar_Man,"[[0.1774890428, 0.2075652335], [0.1992981601, ...","[[0.0028132621, 0.0056265242, 0.0084397862, 0....","[[0.0, 9.81306e-05, 0.00019626110000000002, 0....","[[0.3193268741, 0.3193268741, 0.3193268741, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Between_The_Bars,"[[0.1942119816, 0.23424351510000002], [0.18667...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 0.0001969289, 0.0004281498, 0.000659370...","[[0.3520191452, 0.7109082670000001, 0.71090826...","[[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0,...",0
2,Blue_Collar_Man,"[[0.14879702990000002, 0.174270759], [0.161238...","[[0.0026977515, 0.0053955031, 0.0080932546, 0....","[[0.0, 0.0001046298, 0.00020925970000000002, 0...","[[0.3615991845, 0.3615991845, 0.3615991845, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Thin_Line_Between_Love_And_Hate,"[[0.23499323760000002, 0.2776502958], [0.25781...","[[0.0010679346, 0.0021358692, 0.0032038038, 0....","[[0.0, 9.177850000000001e-05, 0.0001835571, 0....","[[0.1186341942, 0.1186341942, 0.1186341942, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0,...",0
3,Addicted_To_Love,"[[0.1101803537, 0.127565866], [0.1143794879000...","[[0.0015132986000000001, 0.0030265971000000003...","[[0.0, 6.98168e-05, 0.0001396336, 0.0002094505...","[[0.23833650520000002, 0.23833650520000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Addicted_To_Love,"[[0.1568564542, 0.1837942471], [0.160458898700...","[[0.0028146417, 0.0056292833, 0.008443925, 0.0...","[[0.0, 0.0001569411, 0.00040158530000000003, 0...","[[0.3385326631, 0.7009968308, 0.7009968308, 0....","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0,...",1
4,Addicted_To_Love,"[[0.1568564542, 0.1837942471], [0.160458898700...","[[0.0028146417, 0.0056292833, 0.008443925, 0.0...","[[0.0, 0.0001569411, 0.00040158530000000003, 0...","[[0.3385326631, 0.7009968308, 0.7009968308, 0....","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0,...",Take_Me_To_The_River,"[[0.2802345084, 0.32622142400000004], [0.28140...","[[0.0020062926, 0.0040125852, 0.0060188778, 0....","[[0.0, 0.0001100469, 0.0002200938, 0.000330140...","[[0.0952615348, 0.0952615348, 0.0952615348, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Walking_After_Midnight,"[[0.14295036120000001, 0.1690991199], [0.13819...","[[0.0024385958000000003, 0.0048771916000000005...","[[0.0, 8.90445e-05, 0.000178089, 0.0002671335,...","[[0.3674337802, 0.3674337802, 0.3674337802, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0,...",Walking_After_Midnight,"[[0.1728400936, 0.2296872968], [0.1898410711, ...","[[0.0012817682, 0.0025635364, 0.0038453046, 0....","[[0.0, 7.47214e-05, 0.00014944290000000002, 0....","[[0.0981810649, 0.0981810649, 0.0981810649, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",1
150,Train_In_Vain,"[[0.2281798865, 0.26135474140000003], [0.20392...","[[0.0027131144, 0.0054262288, 0.0081393432, 0....","[[0.0, 0.00018822630000000002, 0.0003764526000...","[[0.0, 0.0, 0.0, 0.0, 0.3608843467, 0.36088434...","[[1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",Train_In_Vain,"[[0.109172663, 0.1442114243], [0.1098698720000...","[[0.0015230753000000001, 0.0030461506000000003.

#Mutual features for train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

def data_preprocessing(data):
  data_scaled = gdr.DiagramSelector(use=True, point_type="finite").fit_transform(data)
  data_scaled = gdr.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())]).fit_transform(data_scaled)
  data_scaled = gdr.ProminentPoints(use=True, num_pts=25).fit_transform(data_scaled)
  return data_scaled

In [ ]:
def norm(data1, data2):

  return np.linalg.norm(np.array(data1)-np.array(data2)) 

In [ ]:
def get_mutual_features(pairs_dict):
  pairs_features_dict = {}
  for name, df in tqdm(pairs_dict.items(),total=len(pairs_dict)):

    if 'DTM' in name:
      continue
    pairs_features_dict.setdefault(name,{})
    feature_dict = {}
    df_size = df.shape[0]
    #finding persistence column
    df_persistence_column = [col for col in df.columns if 'persistence' in col]

    for index, row in df.iterrows():

      diag_X = [np.array(row[df_persistence_column[0]])]
      diag_Y = [np.array(row[df_persistence_column[1]])]
    
      #preprocessing
      diag_X = data_preprocessing(diag_X)
      diag_Y = data_preprocessing(diag_Y)


      #computing norms
      #landscape
      ls_norm = norm(row['Landscape_x'], row['Landscape_y'])

      #silhouette norm 2
      sh_norm = norm(row['Silhouette_x'], row['Silhouette_y'])

      #entropy
      et_norm = norm(row['Entropy_vector_x'], row['Entropy_vector_y'])
      #Betti curves
      bc_norm = norm(row['Betti_curves_x'], row['Betti_curves_y'])

      #distances
      #bottleneck
      BD = gdr.BottleneckDistance(epsilon=.001)
      BD.fit(diag_X)
      bd = BD.transform(diag_Y)

      #wasserstein
      WD = gdr.WassersteinDistance(internal_p=2, order=2)
      WD.fit(diag_X)
      wd = WD.transform(diag_Y)

      #kernels
      ##PWG 
      PWG = gdr.PersistenceWeightedGaussianKernel(bandwidth=0.01, kernel_approx=None,\
                                          weight=lambda x: np.arctan(np.power(x[1], 1)))
      PWG.fit(diag_X)
      pwg = PWG.transform(diag_Y)

      ##PSS
      PSS = gdr.PersistenceScaleSpaceKernel(bandwidth=1.)
      PSS.fit(diag_X)
      pss = PSS.transform(diag_Y)

      ##PF
      PF = gdr.PersistenceFisherKernel(bandwidth_fisher=1., bandwidth=1.)
      PF.fit(diag_X)
      pf = PF.transform(diag_Y)

      ##SW
      SW = gdr.SlicedWassersteinKernel(bandwidth=.1, num_directions=100)
      SW.fit(diag_X)
      sw = SW.transform(diag_Y)

      #adding to dict
      feature_dict.setdefault(index, {})
      feature_dict[index]['Landscape_norm'] = ls_norm

      feature_dict[index]['Silhouette_norm'] = sh_norm

      feature_dict[index]['Entropy_norm1'] = et_norm

      feature_dict[index]['Betti_curves_norm'] = bc_norm

      feature_dict[index]['PWG_kernel'] = pwg[0][0]
      feature_dict[index]['PSS_kernel'] = pss[0][0]
      feature_dict[index]['PF_kernel'] = pf[0][0]
      feature_dict[index]['SW_kernel'] = sw[0][0]

      feature_dict[index]['Bottleneck_distance'] = bd[0][0]
      feature_dict[index]['Wasserstein_distance'] = wd[0][0]
      
      dict_df = pd.DataFrame.from_dict(feature_dict).transpose()
      joined_df = df.join(dict_df)
      pairs_features_dict[name] = joined_df
  print('Done!')
  return pairs_features_dict

In [ ]:
features_pair_dict = get_mutual_features(pairs_df_dict)

100%|██████████| 17/17 [00:24<00:00,  1.47s/it]

Done!


In [ ]:
features_pair_dict['tf_sparse']

,name_x,sparse_data_persistence_x,Landscape_x,Silhouette_x,Entropy_vector_x,Betti_curves_x,Entropy_scalar_x,name_y,sparse_data_persistence_y,Landscape_y,Silhouette_y,Entropy_vector_y,Betti_curves_y,Entropy_scalar_y,ref_cover,Landscape_norm,Silhouette_norm,Entropy_norm1,Betti_curves_norm,PWG_kernel,PSS_kernel,PF_kernel,SW_kernel,Bottleneck_distance,Wasserstein_distance
0,Addicted_To_Love,"[[0.1762515338, 0.18988114490000002], [0.27586...","[[0.0028227816, 0.005645563100000001, 0.003009...","[[0.0, 1.47123e-05, 2.94246e-05, 1.54450000000...","[[0.056386639, 0.056386639, 0.056386639, 0.056...","[[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,...",[3.3544529009],Addicted_To_Love,"[[0.2538060425, 0.2684099784], [0.2641463307, ...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 1.95542e-05, 3.91084e-05, 5.86625e-05, ...","[[0.1388507135, 0.1388507135, 0.1388507135, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0,...",[6.2027297306],1,3.496450,1.576296,15.947219,47.169906,7.509133,1.228491,0.999582,1.331143e-04,0.268746,0.428281
1,Addicted_To_Love,"[[0.1762515338, 0.18988114490000002], [0.27586...","[[0.0028227816, 0.005645563100000001, 0.003009...","[[0.0, 1.47123e-05, 2.94246e-05, 1.54450000000...","[[0.056386639, 0.056386639, 0.056386639, 0.056...","[[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,...",[3.3544529009],Blue_Collar_Man,"[[0.3532703957, 0.3566803801], [0.293821828, 0...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 0.0001333848, 0.0002894055, 0.000445426...","[[0.2471316096, 0.5102672398, 0.5102672398, 0....","[[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[4.6328530407],0,1.789406,0.596889,13.421133,43.611925,8.703030,1.473461,0.999095,4.134670e-04,0.168746,0.408416
2,Addicted_To_Love,"[[0.2538060425, 0.2684099784], [0.2641463307, ...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 1.95542e-05, 3.91084e-05, 5.86625e-05, ...","[[0.1388507135, 0.1388507135, 0.1388507135, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0,...",[6.2027297306],Summertime_Blues,"[[0.1604863533, 0.2190179399], [0.2095921893, ...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 7.42998e-05, 0.0001485996, 0.0002228994...","[[0.2637661872, 0.2637661872, 0.2637661872, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[5.0136130408],0,1.602006,0.302662,11.409448,36.783148,12.346404,1.232805,0.999707,1.194147e-03,0.130437,0.310734
3,Between_The_Bars,"[[0.1511386156, 0.1652863275], [0.1711266757, ...","[[0.0028227816, 0.005645563100000001, 0.003886...","[[0.0, 2.26638e-05, 4.53276e-05, 3.08070000000...","[[0.0989793692, 0.0989793692, 0.0989793692, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0,...",[3.9102901065],Between_The_Bars,"[[0.14675995020000002, 0.1647545657], [0.17481...","[[0.0028227816, 0.005645563100000001, 0.006095...","[[0.0, 1.8487600000000002e-05, 3.6975100000000...","[[0.0710891012, 0.0710891012, 0.0710891012, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0,...",[3.2661739749],1,3.107715,1.228937,9.319651,26.720778,4.529648,0.578132,0.999775,1.865040e-03,0.233617,0.340149
4,Between_The_Bars,"[[0.14675995020000002, 0.1647545657], [0.17481...","[[0.0028227816, 0.005645563100000001, 0.006095...","[[0.0, 1.8487600000000002e-05, 3.6975100000000...","[[0.0710891012, 0.0710891012, 0.0710891012, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0,...",[3.2661739749],Time,"[[0.23845018440000001, 0.2440636965], [0.22981...","[[0.0028227816, 0.005645563100000001, 0.008468...","[[0.0, 0.00011963450000000001, 0.0002392690000...","[[0.3512839169, 0.3512839169, 0.3512839169, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[5.7775006988],0,3.549669,1.403403,14.734424,46.572524,4.435556,0.976906,0.999822,3.405455e-04,0.285987,0.407880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,White_Room,"[[0.3037425147, 0.3051978112], [0.320621923100...","[[0.0027614625000000003, 0.005522925000000001,..."

# Mutual features for test

In [ ]:
def get_mutual_features_test(train_test_dict):
  pairs_features_dict = {}
  true_indices = {}
  for name, df_test in train_test_dict.items():
    df = df_test['test']
    if 'DTM' in name:
      continue
    pairs_features_dict.setdefault(name,{})
    true_indices.setdefault(name,{})
    df_query = df.drop_duplicates(subset=['name'])
    df_search = df[~df.index.isin(df_query.index)].drop_duplicates(subset=['name'])

    pers_col = next(col for col in df_query.columns if 'persistence' in col)
    for index_q, row_q in tqdm(df_query.iterrows(), total=df_query.shape[0], desc = 'Progress'):
      search_index = df_search.index[df_query['name'] == row_q['name']][0]
      true_indices[name][index_q] = search_index
      feature_dict = {}
      diag_query = np.array(row_q[pers_col])
      diag_X = data_preprocessing([diag_query])
      for index_s, row_s in df_search.iterrows():
        diag_search = np.array(row_s[pers_col])
        diag_Y = data_preprocessing([diag_search])

        #computing norms
        #landscape
        ls_norm = norm(row_q['Landscape'], row_s['Landscape'])

        #silhouette norm 2
        sh_norm = norm(row_q['Silhouette'], row_s['Silhouette'])

        #entropy
        et_norm = norm(row_q['Entropy_vector'], row_s['Entropy_vector'])
        #Betti curves
        bc_norm = norm(row_q['Betti_curves'], row_s['Betti_curves'])

        #distances
        #bottleneck
        BD = gdr.BottleneckDistance(epsilon=.001)
        BD.fit(diag_X)
        bd = BD.transform(diag_Y)

        #wasserstein
        WD = gdr.WassersteinDistance(internal_p=2, order=2)
        WD.fit(diag_X)
        wd = WD.transform(diag_Y)

        #kernels
        ##PWG 
        PWG = gdr.PersistenceWeightedGaussianKernel(bandwidth=0.01, kernel_approx=None,\
                                            weight=lambda x: np.arctan(np.power(x[1], 1)))
        PWG.fit(diag_X)
        pwg = PWG.transform(diag_Y)

        ##PSS
        PSS = gdr.PersistenceScaleSpaceKernel(bandwidth=1.)
        PSS.fit(diag_X)
        pss = PSS.transform(diag_Y)

        ##PF
        PF = gdr.PersistenceFisherKernel(bandwidth_fisher=1., bandwidth=1.)
        PF.fit(diag_X)
        pf = PF.transform(diag_Y)

        ##SW
        SW = gdr.SlicedWassersteinKernel(bandwidth=.1, num_directions=100)
        SW.fit(diag_X)
        sw = SW.transform(diag_Y)

        #adding to dict
        feature_dict.setdefault(index_s, {})
        feature_dict[index_s]['Landscape_norm'] = ls_norm

        feature_dict[index_s]['Silhouette_norm'] = sh_norm

        feature_dict[index_s]['Entropy_norm1'] = et_norm

        feature_dict[index_s]['Betti_curves_norm'] = bc_norm

        feature_dict[index_s]['PWG_kernel'] = pwg[0][0]
        feature_dict[index_s]['PSS_kernel'] = pss[0][0]
        feature_dict[index_s]['PF_kernel'] = pf[0][0]
        feature_dict[index_s]['SW_kernel'] = sw[0][0]

        feature_dict[index_s]['Bottleneck_distance'] = bd[0][0]
        feature_dict[index_s]['Wasserstein_distance'] = wd[0][0]
      dict_df = pd.DataFrame.from_dict(feature_dict).transpose()
      pairs_features_dict[name][index_q] = dict_df
  print('Done!')
  return pairs_features_dict, true_indices


In [ ]:
test_df, true_indices = get_mutual_features_test(train_test_dict)

Progress: 100%|██████████| 12/12 [00:01<00:00, 11.41it/s]

Done!


In [ ]:
test_df['ts_orig'].keys()

dict_keys([8, 36, 60, 69, 83, 99, 101, 115, 133, 140, 148, 158])

In [ ]:
true_indices['ts_orig']

{8: 9,
 36: 37,
 60: 61,
 69: 70,
 83: 84,
 99: 100,
 101: 102,
 115: 116,
 133: 134,
 140: 141,
 148: 149,
 158: 159}

#Models

In [ ]:
#models
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics, model_selection, svm
from sklearn import preprocessing

def SVM_eval(X_train, X_test, y_train):
  svm = make_pipeline(preprocessing.StandardScaler(), SVC(gamma='auto', probability=True, random_state=42))
  svm.fit(X_train, y_train)
  y_pred = svm.predict_proba(X_test)
  return y_pred[:, 1], svm

def GradBoost_eval(X_train, X_test, y_train):
  clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=42)
  clf.fit(X_train, y_train)
  y_pred = clf.predict_proba(X_test)
  return y_pred[:, 1], clf

def RF_eval(X_train, X_test, y_train):
  clf = RandomForestClassifier(n_estimators=100,
                                 max_depth=5, random_state=42)
  clf.fit(X_train, y_train)
  y_pred = clf.predict_proba(X_test)
  return y_pred[:, 1], clf

#Sequential Feature Selection

In [ ]:
# !pip uninstall scikit-learn -y

# !pip install -U scikit-learn

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.svm import SVC
features=['Landscape_norm',	
          'Silhouette_norm',
          'Entropy_norm1',
          'Betti_curves_norm',
          'PWG_kernel',
          'PSS_kernel',
          'PF_kernel',
          'SW_kernel',
          'Bottleneck_distance',
          'Wasserstein_distance']

def find_features_sequence(df_dict, all_features, est, n_features=3):
  features_dict = {}
  for name, df in df_dict.items():
    X = df[all_features]
    y = df['ref_cover']
    sfs_selector = SequentialFeatureSelector(estimator=est, 
                                             n_features_to_select = n_features, 
                                             cv =10, direction ='backward')
    sfs_selector.fit(X, y)
    X.columns[sfs_selector.get_support()]
    features_dict[name] = X.columns[sfs_selector.get_support()].tolist()
  return features_dict

In [ ]:
est = SVC(kernel="linear", C=1)
seq_features_SVM = find_features_sequence(features_pair_dict, features, est)

In [ ]:
est = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0)
seq_features_GB = find_features_sequence(features_pair_dict, features, est)

In [ ]:
est = RandomForestCl
assifier(n_estimators=100, max_depth=5, random_state=0)
seq_features_RF = find_features_sequence(features_pair_dict, features, est)

KeyboardInterrupt: ignored

## Variance Threshold

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

def find_features_var_thr(df_dict, all_features, trh=1):
  scaler = StandardScaler()
  selector = VarianceThreshold()
  features_dict = {}
  for name, train_test in df_dict.items():
    X = train_test[all_features]
    X_scl = pd.DataFrame(scaler.fit_transform(X), columns = all_features)
      
    selector.fit(X_scl)
    features_dict[name] = X_scl.columns[selector.get_support()].tolist()
  return features_dict

In [ ]:
vt_features = find_features_var_thr(features_pair_dict, features)

#Selected features testing

In [ ]:
def check_ranking_metrics(train_df, test_df, features_dict, true_indices, precision_val=3, model_name='SVM'):
  model_results = {}
  for name, df1 in train_df.items():
    sum_rank = 0
    prec = 0
    used_features = features_dict[name]
    X_train = df1[used_features].to_numpy()
    y_train = df1[['ref_cover']].to_numpy().ravel()
    for indx, df in test_df[name].items():
      X_test = df[used_features].to_numpy()
      if model_name == 'SVM':
        y_pred, clf = SVM_eval(X_train, X_test, y_train)
      elif model_name == 'GradBoost':
        y_pred, clf = GradBoost_eval(X_train, X_test, y_train)
      if model_name == 'RF':
        y_pred, clf = RF_eval(X_train, X_test, y_train)
      n = len(y_pred)
      temp = y_pred.argsort()
      ranks = np.empty_like(temp)
      ranks[temp] = np.arange(n)
      true_idx = [1 if idx == true_indices[name][indx] else 0 for idx in df.index].index(1)
      rank = n - ranks[true_idx]
      sum_rank += rank
      if rank <= 3:
        prec += 1
    prec /= n
    sum_rank /= n
    model_results.setdefault(name, {})
    model_results[name]['MR1'] = sum_rank
    model_results[name]['mAP'] = 1/sum_rank
    model_results[name]['P@'+str(3)] = prec
  return pd.DataFrame.from_dict(model_results).transpose()

In [ ]:
check_ranking_metrics(features_pair_dict, test_df, seq_features_SVM, true_indices)

,MR1,mAP,P@3
af_orig,6.416667,0.155844,0.250000
af_sparse,7.000000,0.142857,0.083333
tf_orig,6.500000,0.153846,0.250000
tf_sparse,6.833333,0.146341,0.166667
tf_sparse_opt,7.666667,0.130435,0.166667
mf_orig,6.333333,0.157895,0.333333
mf_sparse,5.416667,0.184615,0.500000
as_orig,6.916667,0.144578,0.083333
ts_orig,6.000000,0.166667,0.333333
ts_opt,6.583333,0.151899,0.250000


In [ ]:
check_ranking_metrics(features_pair_dict, test_df, seq_features_GB, true_indices, model_name='GradBoost')

,MR1,mAP,P@3
af_orig,6.750000,0.148148,0.416667
af_sparse,6.416667,0.155844,0.250000
tf_orig,7.416667,0.134831,0.083333
tf_sparse,7.916667,0.126316,0.166667
tf_sparse_opt,6.083333,0.164384,0.333333
mf_orig,6.250000,0.160000,0.166667
mf_sparse,5.333333,0.187500,0.500000
as_orig,7.250000,0.137931,0.166667
ts_orig,8.583333,0.116505,0.083333
ts_opt,6.666667,0.150000,0.166667


In [ ]:
check_ranking_metrics(features_pair_dict, test_df, seq_features_RF, true_indices, model_name='RF')

NameError: ignored

In [ ]:
check_ranking_metrics(features_pair_dict, test_df, vt_features, true_indices, model_name='SVM')

,MR1,mAP,P@3
af_orig,7.666667,0.130435,0.166667
af_sparse,6.333333,0.157895,0.083333
tf_orig,7.750000,0.129032,0.083333
tf_sparse,5.666667,0.176471,0.333333
tf_sparse_opt,6.250000,0.160000,0.250000
mf_orig,6.833333,0.146341,0.250000
mf_sparse,7.333333,0.136364,0.166667
as_orig,6.166667,0.162162,0.166667
ts_orig,5.083333,0.196721,0.333333
ts_opt,6.416667,0.155844,0.166667


In [ ]:
check_ranking_metrics(features_pair_dict, test_df, vt_features, true_indices, model_name='RF')

,MR1,mAP,P@3
af_orig,5.416667,0.184615,0.250000
af_sparse,6.416667,0.155844,0.250000
tf_orig,5.750000,0.173913,0.333333
tf_sparse,6.666667,0.150000,0.250000
tf_sparse_opt,5.833333,0.171429,0.333333
mf_orig,7.583333,0.131868,0.166667
mf_sparse,5.750000,0.173913,0.333333
as_orig,6.916667,0.144578,0.083333
ts_orig,8.166667,0.122449,0.166667
ts_opt,6.916667,0.144578,0.166667


In [ ]:
check_ranking_metrics(features_pair_dict, test_df, vt_features, true_indices, model_name='GradBoost')

,MR1,mAP,P@3
af_orig,5.083333,0.196721,0.333333
af_sparse,6.750000,0.148148,0.333333
tf_orig,6.250000,0.160000,0.333333
tf_sparse,6.333333,0.157895,0.250000
tf_sparse_opt,6.250000,0.160000,0.250000
mf_orig,5.666667,0.176471,0.416667
mf_sparse,6.916667,0.144578,0.250000
as_orig,7.666667,0.130435,0.166667
ts_orig,8.166667,0.122449,0.000000
ts_opt,6.333333,0.157895,0.333333
